In [4]:
%%capture
import sys
import numpy as np

%pip install pyomo >/dev/null 2>/dev/null
%pip install highspy >/dev/null 2>/dev/null

solver = 'appsi_highs'

import pyomo.environ as pyo
SOLVER = pyo.SolverFactory(solver)

assert SOLVER.available(), f"Solver {solver} is not available."

import pandas as pd

In [5]:
print('Pyomo and Highs installed successfully.')

Pyomo and Highs installed successfully.


In [6]:
m = pyo.ConcreteModel("National Park Trip")

# Sets
activity_table = pd.read_excel("data/activities.xlsx", sheet_name="Activities") # use correct path
activity_table = activity_table.set_index("Activity")


FileNotFoundError: [Errno 2] No such file or directory: 'data/activities.xlsx'